# Import Libraries

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.framework import convert_to_constants
from tensorflow.keras.preprocessing import image

# Define

In [2]:
def get_dummy_images(batch_size = 32, img_shape = [224, 224, 3]):
    img=tf.random.uniform(shape=[batch_size] + img_shape, dtype=tf.float32)
    print("Generated input random images with shape (N, H, W, C) =", img.shape)
    return img

# converting

- reference: https://docs.nvidia.com/deeplearning/frameworks/tf-trt-user-guide/index.html


-----


TF-TRT 는 Tensorflow 에서 생성된 saved model 을 사용하여 TensorRT 에 최적화된 saved model 을 생성한다. 
- `f32_trt_convert` 함수는 모델을 float32 정밀도로 변환한다.
- `f16_trt_convert` 함수는 모델을 float32 정밀도로 변환한다.
- `int8_trt_convert` 함수는 모델을 float32 정밀도로 변환한다. 

In [3]:
def f32_trt_convert(input_path, output_path, input_shapes, explicit_batch=False):
    conv_params=trt.TrtConversionParams(
        precision_mode=trt.TrtPrecisionMode.FP32)
    
    converter = trt.TrtGraphConverterV2(
        input_saved_model_dir=input_path, 
        conversion_params=conv_params,
        use_dynamic_shape=explicit_batch,
        dynamic_shape_profile_strategy='Optimal')

    converter.convert()
    
    def input_fn():
        for shapes in input_shapes:
            # return a list of input tensors
            yield [np.ones(shape=x).astype(np.float32) for x in shapes]

    converter.build(input_fn)
    converter.save(output_path)

In [4]:
def f16_trt_convert(input_path, output_path, input_shapes, explicit_batch=False):
    
    conv_params = trt.TrtConversionParams(
        precision_mode=trt.TrtPrecisionMode.FP16)
    
    converter = trt.TrtGraphConverterV2(
       input_saved_model_dir = input_path, 
        conversion_params = conv_params)

    # def input_fn():
    #     for shapes in input_shapes:
    #         # return a list of input tensors
    #         yield [np.ones(shape=x).astype(np.float32) for x in shapes]

    converter.convert()
    
    # converter.build(input_fn)
    converter.save(output_path)

In [5]:
def int8_trt_convert(input_path, output_path, input_shapes, explicit_batch=False):
    conv_params=trt.TrtConversionParams(
        precision_mode=trt.TrtPrecisionMode.INT8,
        use_calibration=False)
    
    converter = trt.TrtGraphConverterV2(
        input_saved_model_dir=input_path, 
        conversion_params=conv_params,
        use_dynamic_shape=explicit_batch,
        dynamic_shape_profile_strategy='Optimal')

    converter.convert()
    
    def input_fn():
        for shapes in input_shapes:
            # return a list of input tensors
            yield [np.ones(shape=x).astype(np.float32) for x in shapes]

    converter.build(input_fn)
    converter.save(output_path)

In [6]:
input_data =  get_dummy_images(batch_size = 1, img_shape = [224, 224, 3])

Generated input random images with shape (N, H, W, C) = (1, 224, 224, 3)


In [7]:
# measure TF-TRT converting time
f32_trt_convert(input_path='../saved_models/custom_saved_model', 
                 output_path='custom_saved_model_TFTRT_F32', 
                 input_shapes = [[input_data.shape]],
                 explicit_batch=False)

INFO:tensorflow:Linked TensorRT version: (8, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (8, 0, 1)
INFO:tensorflow:Assets written to: custom_saved_model_TFTRT_F32/assets


In [8]:
# measure TF-TRT converting time
int8_trt_convert(input_path='../saved_models/custom_saved_model', 
                 output_path='custom_saved_model_TFTRT_INT8', 
                 input_shapes = [[input_data.shape]],
                 explicit_batch=False)

INFO:tensorflow:Linked TensorRT version: (8, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (8, 0, 1)
INFO:tensorflow:Assets written to: custom_saved_model_TFTRT_INT8/assets


In [ ]:
# measure TF-TRT converting time
f32_trt_convert(input_path='../saved_models/custom_resnet_saved_model', 
                 output_path='custom_resnet_saved_model_TFTRT_F32', 
                 input_shapes = [[input_data.shape]],
                 explicit_batch=False)

In [ ]:
# measure TF-TRT converting time
int8_trt_convert(input_path='../saved_models/custom_resnet_saved_model', 
                 output_path='custom_resnet_saved_model_TFTRT_INT8', 
                 input_shapes = [[input_data.shape]],
                 explicit_batch=False)

In [14]:
# measure TF-TRT converting time
f32_trt_convert(input_path='../saved_models/custom_mobilenetv2_saved_model', 
                 output_path='custom_mobilenetv2_saved_model_TFTRT_F32', 
                 input_shapes = [[input_data.shape]],
                 explicit_batch=False)

INFO:tensorflow:Linked TensorRT version: (8, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (8, 0, 1)
INFO:tensorflow:Assets written to: custom_mobilenetv2_saved_model_TFTRT_F32/assets


In [9]:
# measure TF-TRT converting time
int8_trt_convert(input_path='../saved_models/custom_mobilenetv2_saved_model', 
                 output_path='custom_mobilenetv2_saved_model_TFTRT_INT8', 
                 input_shapes = [[input_data.shape]],
                 explicit_batch=False)

INFO:tensorflow:Linked TensorRT version: (8, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (8, 0, 1)
INFO:tensorflow:Assets written to: custom_mobilenetv2_saved_model_TFTRT_INT8/assets


In [15]:
# measure TF-TRT converting time
f32_trt_convert(input_path='../saved_models/custom_efficientnetb2_saved_model', 
                 output_path='custom_efficientnetb2_saved_model_TFTRT_F32', 
                 input_shapes = [[input_data.shape]],
                 explicit_batch=False)

INFO:tensorflow:Linked TensorRT version: (8, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (8, 0, 1)


INFO:tensorflow:Assets written to: custom_efficientnetb2_saved_model_TFTRT_F32/assets


INFO:tensorflow:Assets written to: custom_efficientnetb2_saved_model_TFTRT_F32/assets


In [ ]:
# measure TF-TRT converting time
int8_trt_convert(input_path='../saved_models/custom_efficientnetb2_saved_model', 
                 output_path='custom_efficientnetb2_saved_model_TFTRT_INT8', 
                 input_shapes = [[input_data.shape]],
                 explicit_batch=False)

In [ ]:
# measure TF-TRT converting time
f32_trt_convert(input_path='../saved_models/custom_efficientnetb1_saved_model', 
                 output_path='custom_efficientnetb1_saved_model_TFTRT_F32', 
                 input_shapes = [[input_data.shape]],
                 explicit_batch=False)

In [ ]:
# measure TF-TRT converting time
int8_trt_convert(input_path='../saved_models/custom_efficientnetb1_saved_model', 
                 output_path='custom_efficientnetb1_saved_model_TFTRT_INT8', 
                 input_shapes = [[input_data.shape]],
                 explicit_batch=False)

In [12]:
saved_model_infer['construction_danger']

[['construction_danger',
  'park_danger',
  array([[9.9902344e-01, 6.9570541e-04, 0.0000000e+00, 0.0000000e+00,
          3.9815903e-05, 2.3663044e-05]], dtype=float32)],
 ['construction_danger',
  'park_danger',
  array([[9.9902344e-01, 6.9570541e-04, 0.0000000e+00, 0.0000000e+00,
          3.9815903e-05, 2.3663044e-05]], dtype=float32)],
 ['construction_danger',
  'park_danger',
  array([[9.9902344e-01, 6.9570541e-04, 0.0000000e+00, 0.0000000e+00,
          3.9815903e-05, 2.3663044e-05]], dtype=float32)],
 ['construction_danger',
  'park_danger',
  array([[9.9902344e-01, 6.9570541e-04, 0.0000000e+00, 0.0000000e+00,
          3.9815903e-05, 2.3663044e-05]], dtype=float32)],
 ['construction_danger',
  'park_danger',
  array([[9.9902344e-01, 6.9570541e-04, 0.0000000e+00, 0.0000000e+00,
          3.9815903e-05, 2.3663044e-05]], dtype=float32)],
 ['construction_danger',
  'park_danger',
  array([[9.9902344e-01, 6.9570541e-04, 0.0000000e+00, 0.0000000e+00,
          3.9815903e-05, 2.366304

In [13]:
saved_model_infer['construction_safe']

[['construction_safe',
  'park_safe',
  array([[2.8002930e-01, 7.1777344e-01, 2.3841858e-07, 1.3709068e-06,
          7.0810318e-05, 2.1705627e-03]], dtype=float32)],
 ['construction_safe',
  'park_safe',
  array([[2.8002930e-01, 7.1777344e-01, 2.3841858e-07, 1.3709068e-06,
          7.0810318e-05, 2.1705627e-03]], dtype=float32)],
 ['construction_safe',
  'park_safe',
  array([[2.8002930e-01, 7.1777344e-01, 2.3841858e-07, 1.3709068e-06,
          7.0810318e-05, 2.1705627e-03]], dtype=float32)],
 ['construction_safe',
  'park_safe',
  array([[2.8002930e-01, 7.1777344e-01, 2.3841858e-07, 1.3709068e-06,
          7.0810318e-05, 2.1705627e-03]], dtype=float32)],
 ['construction_safe',
  'park_safe',
  array([[2.8002930e-01, 7.1777344e-01, 2.3841858e-07, 1.3709068e-06,
          7.0810318e-05, 2.1705627e-03]], dtype=float32)],
 ['construction_safe',
  'park_safe',
  array([[2.8002930e-01, 7.1777344e-01, 2.3841858e-07, 1.3709068e-06,
          7.0810318e-05, 2.1705627e-03]], dtype=float32)]

In [4]:
efficientnetb1_saved_model_loaded = tf.keras.models.load_model('custom_efficientnetb1_saved_model_TFTRT_INT8')

In [7]:
efficientnetb1_saved_model_infer = {}
for i in class_names:
    efficientnetb1_saved_model_infer[i] = infer(i, efficientnetb1_saved_model_loaded)

In [8]:
efficientnetb1_saved_model_infer

{'park_danger': [['park_danger',
   'road_danger',
   array([[1.7387732e-07, 2.7159424e-09, 9.9842691e-01, 8.4345320e-06,
           1.2884867e-06, 1.5632474e-03]], dtype=float32)],
  ['park_danger',
   'road_danger',
   array([[1.7387732e-07, 2.7159424e-09, 9.9842691e-01, 8.4345320e-06,
           1.2884867e-06, 1.5632474e-03]], dtype=float32)],
  ['park_danger',
   'road_danger',
   array([[1.7387732e-07, 2.7159424e-09, 9.9842691e-01, 8.4345320e-06,
           1.2884867e-06, 1.5632474e-03]], dtype=float32)],
  ['park_danger',
   'road_danger',
   array([[1.7387732e-07, 2.7159424e-09, 9.9842691e-01, 8.4345320e-06,
           1.2884867e-06, 1.5632474e-03]], dtype=float32)],
  ['park_danger',
   'road_danger',
   array([[1.7387732e-07, 2.7159424e-09, 9.9842691e-01, 8.4345320e-06,
           1.2884867e-06, 1.5632474e-03]], dtype=float32)],
  ['park_danger',
   'road_danger',
   array([[1.7387732e-07, 2.7159424e-09, 9.9842691e-01, 8.4345320e-06,
           1.2884867e-06, 1.5632474e-03]], 

In [12]:
efficientnetb1_saved_model_infer['park_danger'][0][1]

'road_danger'

In [9]:
_efficientnetb1_saved_model_infer = {}
for i in class_names:
    _efficientnetb1_saved_model_infer[i] = 0

In [14]:
for label in class_names:
    count = {}
    for i in class_names:
        count[i] = 0
    for res in efficientnetb1_saved_model_infer[label]:
        count[res[1]] += 1
    _efficientnetb1_saved_model_infer[label] = count

In [15]:
_efficientnetb1_saved_model_infer

{'park_danger': {'park_danger': 0,
  'park_safe': 0,
  'road_danger': 10,
  'road_safe': 0,
  'construction_danger': 0,
  'construction_safe': 0},
 'park_safe': {'park_danger': 0,
  'park_safe': 0,
  'road_danger': 10,
  'road_safe': 0,
  'construction_danger': 0,
  'construction_safe': 0},
 'road_danger': {'park_danger': 0,
  'park_safe': 0,
  'road_danger': 0,
  'road_safe': 0,
  'construction_danger': 10,
  'construction_safe': 0},
 'road_safe': {'park_danger': 0,
  'park_safe': 0,
  'road_danger': 10,
  'road_safe': 0,
  'construction_danger': 0,
  'construction_safe': 0},
 'construction_danger': {'park_danger': 10,
  'park_safe': 0,
  'road_danger': 0,
  'road_safe': 0,
  'construction_danger': 0,
  'construction_safe': 0},
 'construction_safe': {'park_danger': 0,
  'park_safe': 10,
  'road_danger': 0,
  'road_safe': 0,
  'construction_danger': 0,
  'construction_safe': 0}}